# Web Scraping con Python
Vamos a usar las librerías [Requests](https://requests.kennethreitz.org/en/master/) y [BeautifulSoup](https://www.crummy.com/software/BeautifulSoup/bs4/doc/) para hacer web scraping. Instalación (en Anaconda):
```
conda install requests beautifulsoup4
```

In [ ]:
import requests
from bs4 import BeautifulSoup
import re

In [ ]:
r = requests.get("https://idal.uv.es/ejemplo.html", params=dict(q="h"))

In [ ]:
type(r)

In [ ]:
r.url

Para obtener la respuesta completa (código HTML de la página) como una cadena de texto accedemos al atributo `text`:

In [ ]:
print(r.text)

El objeto devuelto es de tipo `requests.models.Response` con unos atributos determinados:

In [ ]:
[s for s in dir(r) if not s.startswith('_')]

Podemos ver el código de respuesta del servidor (útil para detectar errores 4XX o 5XX):

In [ ]:
r.status_code

Podemos ver todas las cabeceras de respuesta en caso de respuesta afirmativa:

In [ ]:
r.headers

Para comprobar el tipo de respuesta (Content Type) y ver si es HTML, JSON, XML, etc.:

In [ ]:
r.headers["content-type"]

Si el servidor diera la información de la codificación incorrecta (p. ej. ISO-8859-1 para una página UTF-8) podemos forzar una nueva codificación

In [ ]:
r.encoding

In [ ]:
r.encoding='UTF-8' #cambiar a ISO-8859-1 para comprobar que muestra mal los acentos

### Extraer contenido del HTML
Usar expresiones regulares para buscar patrones HTML no está recomendado. Es mejor usar una librería para "parsear" el HTML y extraer el contenido de las etiquetas específicas.  
Aún así, las RegEx se pueden usar para buscar patrones específicos como listas de precios, direcciones de correo, números de teléfono.  
Podemos ejecutar una RegEx en el texto de respuesta para buscar un patrón específico, como p. ej. una URL:

In [ ]:
import re

re.findall(r'https?://[\w_./]+', r.text)

## Librería BeautifulSoup
La librería BeautifulSoup se utiliza para extraer contenidos desde una página HTML. Tiene una API muy simple a la vez que potente.  
Para empezar, hay que convertir el HTML del texto de respuesta en una estructura DOM jerárquica que se pueda recorrer y buscar.


In [ ]:
soup = BeautifulSoup(r.text, "html.parser")

In [ ]:
type(soup)

Podemos referirnos a cada elemento del DOM por su etiqueta dentro del objeto `soup` (si hay varias apariciones sólo devuelve la primera)

In [ ]:
soup.title

In [ ]:
type(soup.title)

Podemos ver el texto del elemento con el atributo `text`

In [ ]:
titulo = soup.title.text
print(titulo)

In [ ]:
type(titulo)

In [ ]:
soup.a

In [ ]:
soup.a.text

In [ ]:
soup.p

In [ ]:
soup.p.text

In [ ]:
soup.text

Podemos acceder a los atributos de un elemento como un diccionario dentro del elemento

In [ ]:
soup.a.attrs

In [ ]:
soup.a['class']

In [ ]:
soup.a['title']

In [ ]:
soup.div['class']

### Búsqueda de elementos
El método `find_all` devuelve una lista con todas las etiquetas que encuentra. El método `find` sólo devuelve el primer resultado de la búsqueda.   
En ambos casos, cada etiqueta contiene todo su contenido en la estructura DOM.

In [ ]:
link = soup.find("a")
link

In [ ]:
#equivale a
soup.a

EL objeto que devuelve `find()` es del tipo `Tag`


In [ ]:
type(link)

In [ ]:
#atributos y métodos de Tag
[s for s in dir(link) if not s.startswith('_')]

De cada enlace podemos obtener sus atributos

In [ ]:
link.get('class') #equivalente a link['class']

In [ ]:
link.attrs

Podemos anidar varias etiquetas a buscar

In [ ]:
soup.find('p').find('em')

In [ ]:
soup.find("em")

Podemos acotar la búsqueda a una clase específica de la etiqueta (p.ej. `<a class="enlace">...</a>`)

In [ ]:
tag = soup.find("a", class_="enlace")
print(tag)

In [ ]:
#o simplemente
tag = soup.find("a", "enlace")
print(tag)

Podemos buscar una etiqueta con un atributo ID específico (p.ej.: `<div id="col4">...</div>`)

In [ ]:
tag = soup.find("div", id="col4")
print(tag)

Para buscar todas las etiquetas de un tipo en el árbol DOM (por ejemplo los enlaces `<a>` de una página) 

In [ ]:
enlaces = soup.find_all("a")

In [ ]:
enlaces

Devuelve un objeto especial que se comporta como una lista de objetos `Tag`

In [ ]:
type(enlaces)

In [ ]:
[s for s in dir(enlaces) if not s.startswith('_')]

In [ ]:
type(enlaces[0])

El primer elemento devuelto por `find_all` es el mismo que devuelve el atributo de ese elemento en el objeto `soup`

In [ ]:
soup.find_all("div")[0]

In [ ]:
soup.div

In [ ]:
#hay un shortcut para find_all
soup("a")

In [ ]:
#podemos buscar varios tipos de elemento a la vez
soup.find_all(["a", "p"])

In [ ]:
titulos = soup.find_all(re.compile("h\d+"))

In [ ]:
[t.name for t in titulos]

Podemos filtrar por un atributo del Tag (por defecto es la clase si no indicamos otra cosa)

In [ ]:
for link in soup.find_all('a', class_="enlace"):
    print(link)

Como el objeto devuelto por `find_all` y `select` es una lista de `Tags` podemos obtener los atributos de cada uno de los elementos

In [ ]:
for link in soup.find_all('a', class_="enlace"):
    print(link["href"], link["title"])

In [ ]:
enlaces = soup.find_all("a", "btn")
for e in enlaces:
    print(e)

Podemos hacer una búsqueda de otra etiqueta anidada dentro de la primera búsqueda (útil por ejemplo para buscar elementos genéricos dentro de una sección específica de la página)

In [ ]:
tags = soup.find("div", id="col4").find_all("a")
print(tags)

Repetimos esta búsqueda especificando un selector CSS (más simple que lo anterior si se conoce la sintaxis CSS)

In [ ]:
soup.select("a") #equivale a soup.find_all("a")

In [ ]:
soup.select("#col4")

In [ ]:
tags = soup.select("#col4 a")
print(tags)

El atributo `contents` del resultado de una etiqueta individual (objeto `bs4.element.Tag`) contiene una lista de objetos con su contenido interno (que incluye tanto los nodos de texto como la representación e texto de todo el HTML anidado).  
Si el objeto sólo contiene texto es del tipo `NavigableString`, y si contiene una etiqueta es del tipo `Tag`. Los saltos de línea (`\n`) entre las etiquetas se consideran elementos de texto y aparecen en la lista devuelta.

In [ ]:
print(soup.find("title"))

In [ ]:
inner_contents = soup.find("title").contents
print(inner_contents)

In [ ]:
soup.find("div")

In [ ]:
soup.div.p.contents

In [ ]:
for e in soup.div.p.contents:
    print(e, type(e))

También podemos buscar por un atributo en particular del elemento o por una clase.

In [ ]:
soup.find(href="#")

In [ ]:
soup.find(class_="col-sm-4").h3.text

### Ejercicio
Muestra el contenido interno del div `#col4`

El atributo `text` del objeto `bs4.element.Tag` contiene una lista de strings con los textos contenidos en la etiqueta, ignorando todas las etiquetas HTML (como por ejemplo `<span>`, `<strong>` o `<i>`):

In [ ]:
inner_text = soup.find("div", id="col4").text.strip()
print(inner_text)

Aquí el método `strip()` simplemente elimina las líneas en blanco (`\n`) del resultado.  

### Búsqueda por función
Podemos definir una función booleana para hacer la búsqueda de elementos en el árbol DOM

In [ ]:
def etiquetas(tag):
    return tag.has_attr('id')

soup.find_all(etiquetas)

### Ejercicio
Lista todos los Tags que cumplan el filtrado de la función `etiquetas()` y muéstralos como una lista de tuplas `(tag.name, tag.attrs)`

## Navegación por la estructura DOM
Cada elemento `Tag` guarda información de su posición dentro del árbol de etiquetas HTML del documento (estructura DOM) de manera que se puede navegar desde cada etiqueta a sus etiquetas relacionadas:
### Navegar hacia abajo
Podemos usar los nombres de las etiquetas como atributos encadenados del objeto `soup` para recorrer el árbol hacia abajo:

In [ ]:
soup.div.h1 #busca la primera etiqueta <h1> dentro de la primera etiqueta <div> 

In [ ]:
soup("div")[1].h3

Todos los hijos de una etiqueta están en su atributo `contents`:

In [ ]:
soup.div.contents

Las etiquetas que sólo contienen texto aparecen en la lista de `contents` como un objeto `NavigableString`

In [ ]:
soup.div.h1.contents

In [ ]:
type(soup.div.h1.contents[0])

Podemos iterar sobre los descendientes de una etiqueta con el iterador `children`:

In [ ]:
soup.div.children

In [ ]:
[child for child in soup.div.children]

In [ ]:
#la lista que genera children es equivalente a contents
list(soup.div.children)==soup.div.contents

Con `contents` y `children` consideran sólo los descencientes directos de una etiqueta. Con `descendants` accedemos iterativamente a todos sus descendientes:

In [ ]:
soup.div.p

In [ ]:
soup.div.p.descendants

In [ ]:
[child for child in soup.div.p.descendants]

Por contra...

In [ ]:
[child for child in soup.div.p.children]

Para obtener sólo el texto de una etiqueta usamos el atributo `string`, o para obtener iterativamente todos los textos contenidos usamos el iterador `strings`:

In [ ]:
soup.div.h1.string

In [ ]:
#si una etiqueta contiene otras etiquetas el atributo string está vacío
soup.div.p

In [ ]:
soup.div.p.string

In [ ]:
#iteramos por todos los elementos contenidos para obtener su string
[s for s in soup.div.p.strings]

### Navegar hacia arriba
Con `parent` obtenemos la etiqueta superior a una dada, y con `parents` obtenemos iterativamente todos sus ancentros:

In [ ]:
soup.div.h1.parent

In [ ]:
soup.div.h1.parents

### Ejercicio
Busca el ascendente directo del div `#col4`

### Ejercicio
Itera sobre los ancestros del div `#col4` y muestra cada uno de sus nombres y atributos

### Navegar hacia los lados
Los métodos `next_sibling` y `previous_sibling` acceden a la etiqueta posterior o anterior del mismo nivel. Los métodos `next_siblings` y `previous_siblings` lo hacen iterativamente

In [ ]:
soup.find("div", id="enlaces")

In [ ]:
soup.find("div", id="enlaces").ul.contents

In [ ]:
soup.find("div", id="enlaces").li.previous_sibling

In [ ]:
soup.find("div", id="enlaces").li.next_sibling.next_sibling

In [ ]:
soup.find("div", id="enlaces").find("a",{"title": "ETSE"}).parent

In [ ]:
#buscamos elementos hermanos a continuación del <li> intermedio
enlace_etse = soup.find("div", id="enlaces").find("a",{"title": "ETSE"})
[tag for tag in enlace_etse.parent.next_siblings]

In [ ]:
#buscamos elementos hermanos anteriores al <li> final

[tag for tag in soup.find("div", id="enlaces").find("a",{"title": "UV"}).parent.previous_siblings]

### Ejemplo completo
Del siguiente código HTML vamos a extraer los ítems listados y su precio

In [ ]:
texto = """
<body>
<div id="listings_prices">
 <div class="item">
  <li class="item_name">Reloj</li>
  <div class="main_price">Precio: 66.68€</div>
  <div class="discounted_price">Precio con descuento: 46.68€</div>
   </div>
   <div class="item">
  <li class="item_name">Móvil</li>
  <div class="main_price">Precio: 566.68€</div>
   </div>
</div>
</body>
"""

In [ ]:
soup = BeautifulSoup(texto, "html.parser")
for product in soup.find_all("div", "item"):
    product_title = product.find("li").text 
    product_price = re.search(r'\d+\.\d+€', product.find("div", "main_price").text)[0]
    print(f"{product_title} se vende por {product_price}")

### Ejercicio
¿Cuál es el precio con descuento de los ítems?